<a href="https://colab.research.google.com/github/CookiesAndWater/MAT_422/blob/main/MAT_422_4_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 4 Network Analysis

# 4.1.1 Graph Models

* **Graph Models**: Consist of objects called nodes and the connections between them called edges.
  * In notation, denoted as $G(V,E)$, where $V$ is the set of nodes and $E$ is the set of edges. $m=|E|$ represents the number of edges
* **Edge Notation**: Edges can be represented between their endpoints: $e(v_1,v_2)$ or $(v_1,v_2)$, where this particular example indicates a connection between node $v_1$ and
  $v_2$
* **Edge Direction**: If only one node is connected to another but not vice versa
  * **Directed Edges**: Have direction, $e(v_1,v_2) \neq e(v_2,v_1)$
    * Graph is called directed graphs
  * **Undirected Edges**: Nodes are connected both ways.
    * Graph is called undirected graphs
  * **Mixed Graphs**: Contain both directed and undirected edges.
* **Path**: A sequence of edges where nodes and edges are distinct
  * **Cycle**: Closed Path
  * **Path Length**: Number of edges in path or cycle
* **Direct Graphs**: Only direct paths are counted following the direction
* **Shortest Path**: What we are interested in, the path with the shortest length. Used as distance for modeling on networks
* **Neighborhood of a node**: A $n$-hop neighborhood of a node $v_i$ is the set of nodes that are withing $n$ hops distance from that node.

* **Node degree**: The number of edges connected to a node.
  * A directed graph has two types:
    * **In-degrees**: Edges pointed towards the node
    * **Out-degrees**: Edges pointed away from the node
* **Centrality**: Nodes with the most connections. The more connections, the more important it is
  * **In-degree Centrality**: Describes the popularity of a node and it's prominence
  * **Out-degree Centrality**: Describes the gregariousness(outwardness) of a node


## 4.1.2 Laplacian Matrix

* **Adjacency Matrix**: Represents a graph with $n$ nodes using an $n × n$ matrix
  * A value of 1 at position $M_{ij}$ indicates an edge between nodes $v_i$ and $v_j$, a value of 0 means no edge
  * When generalized, any real numbers can indicate strength of connection.
  * **Direct/Undirected Graphs**: Directed graphs have non-symmetric adjacency matrices, while Undirected graphs have symmetric adjacency matrices($A=A^T$)
* **Laplacian Matrix**: A symmetric $n \times n$ matrix representing the graph structure
  * **Diagonal entries**: $L_{ii} = ∑_kE_{ik}$ (sum of edge weights connected to vertex $v_i$)
  * **Off-diagonal entries**: $L_{ij} = -E_{ij}$ if $v_i$ and $v_j$ are adjacent, 0 if not adjacent.
* **Incidence Matrix**: A $n \times m$ matrix where $n$ is the number of vertices and $m$ is the number of edges
  * Each edge has two non-zero entries corresponding to its two vertices, $\sqrt{E_{ij}}$ and $-\sqrt{E_{ij}}$

## 4.2 Spectral Graph Bipartitioning

* **Graph Partitioning**: The goal is to partition a graph into two sets, $V_1$ and $V_2$ such that the **cut** (the number of edges bewtween the sets) is minimized
  * The **cut** between $V_1$ and $V_2$ for a weighted graph $G = (V,E)$ is defined as
    * cut$(V_1, V_2) = ∑_{i ∈ V_1, j ∈ V_2}M_i$$_j$
    * The definition of a cut can be extended to k vertex subsets
* **Classical Bipartitioning**: Finding a bipartition where the **cut** is minimized and the set $V_1$ and $V_2$ are nearly equal is size.
  * $p_i = $ \begin{cases}
      +1, & i ∈ V_1 \\
      -1, & i ∈ V_2
      \end{cases}
* **Rayleigh Quotient**: The cut can be minimized using the Rayleigh Quotient involving the **Laplacian matrix** $L$ of the graph
  * $\frac{p^TLp}{p^Tp} = \frac{1}{n}*4cut(V_1,V_2)$
* **Objective Function for balancing**: To balance cuts, we define a weight matrix $W$, where $w_{ii}$ is a weight for each vertex i. We want to minimize the following function
  * $Q(V_1, V_2) = \frac{cut(V_1,V_2)}{Wv_1} + \frac{cut(V_1,V_2)}{Wv_2}$
    * This favors partitions that have a small cut value and are balanced
* **Generalized Partition Vector**:
  * A generalized partition vector $q$ satisfies
    * $q_i$ =  \begin{cases}
      \frac{\sqrt{v_2}}{\sqrt{v1}}, & i ∈ V_1 \\
      -\frac{\sqrt{v_2}}{\sqrt{v1}}, & i ∈ V_2
      \end{cases}
    * The objective function minimization is achieved by solving a generalized eigenvale problem
      * $Lx = λWx$, where the solution is the second smallest eigenvalue $λ_2$
* **Normalized Cut**: Used to penalize unbalanced partitions
  * Normalized Cut$(V_1, V_2) = \frac{cut(V_1, V_2)}{∑_{i∈V_1}w_{ii}} + \frac{cut(V_1, V_2)}{∑_{i∈V_1}w_{ii}}$

In [18]:
import networkx as nx
import numpy as np
import scipy.linalg as linalg
import scipy.sparse as sp
from scipy.sparse.csgraph import laplacian

#Simple Graph
G = nx.erdos_renyi_graph(6,0.5)

#Adjacency matrix
A = nx.to_numpy_array(G)

#Degree matrix
D = np.diag(np.sum(A, axis = 1))

#Laplacian matrix
L = D - A

#Eigenvalues and eigenvectors, Lx = lambda * Wx
eigenvalues, eigenvectors = linalg.eigh(L)
sort_eigenvals = np.argsort(eigenvalues)
eigvals = eigenvalues[sort_eigenvals]
eigvec = eigenvectors[:,sort_eigenvals]

fiedler_vector = eigvec[:,1]

#partitions
partition1 = np.where(fiedler_vector > 0)[0]
partition2 = np.where(fiedler_vector < 0)[0]

print("Partition 1:", partition1)
print("Partition 2:", partition2)

cut_value = np.sum(A[partition1, :][:,partition2])
print("Cut value:", cut_value)

Partition 1: [1 2 4 5]
Partition 2: [3]
Cut value: 3.0


We first make simple graph with the networks library. We then create the **adjacency matrix** using it's built in functions. We then create a **degree matrix** where each entry is the degree of the node. We then get the **laplacian matrix**, with $L = D - A$.  We solve the eigenvalue problem $Lx = λWx$. From here we get the second smallest eigenvector,the **Fiedler vector**, for optimal partitioning. We then make the partitions based on the Fiedler vector. The cut is calculated by summing the weights of the adjacency matrix entries.